<a href="https://colab.research.google.com/github/amimulhasan/Deep-Learning/blob/main/cnn%2Bcapsul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow numpy opencv-python
#

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2


In [ ]:
# from google.colab import files
# files.upload()  # Use this to upload your kaggle.json file
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download deeppythonist/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/deeppythonist/brain-tumor-mri-dataset
License(s): MIT
 74% 112M/151M [00:00<00:00, 1.16GB/s]
100% 151M/151M [00:00<00:00, 975MB/s] 


In [5]:
import zipfile

zip_path = '/content/brain-tumor-mri-dataset.zip'
extract_to = 'Tumor_brain_cancer'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Unzipping completed!")

Unzipping completed!


In [8]:
import numpy as np
from PIL import Image
import os

# ------------------------
# SETTINGS
# ------------------------
dataset_dir = "/content/Tumor_brain_cancer/train"  # Update path as needed
img_size = (128, 128)

X = []
y = []

# --- Get all class folders (e.g., 4 classes) ---
class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
print("✅ Classes found:", class_names)

if len(class_names) != 4:
    raise ValueError(f"Expected exactly 4 classes, found {len(class_names)}: {class_names}")

# --- Walk through each class and collect images ---
for idx, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_dir, class_name)
    print(f"🔍 Processing class '{class_name}' ({idx})")

    for root, _, files in os.walk(class_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0  # Normalize
                X.append(img_array)
                y.append(idx)
            except Exception as e:
                print(f"⚠️ Failed to process {file_path}: {e}")

# ------------------------
# CONVERT TO NUMPY ARRAYS
# ------------------------
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

print(f"\n✅ Dataset loaded successfully:")
print(f"   X shape = {X.shape}")
print(f"   y shape = {y.shape}")
print(f"   Class mapping = {dict(enumerate(class_names))}")

# ------------------------
# SAVE AS .NPY FILES
# ------------------------
np.save("X.npy", X)
np.save("y.npy", y)
print("✅ Saved X.npy and y.npy to current working directory.")


✅ Classes found: ['glioma', 'meningioma', 'notumor', 'pituitary']
🔍 Processing class 'glioma' (0)
🔍 Processing class 'meningioma' (1)
🔍 Processing class 'notumor' (2)
🔍 Processing class 'pituitary' (3)

✅ Dataset loaded successfully:
   X shape = (5723, 128, 128, 3)
   y shape = (5723,)
   Class mapping = {0: 'glioma', 1: 'meningioma', 2: 'notumor', 3: 'pituitary'}
✅ Saved X.npy and y.npy to current working directory.


In [9]:
import numpy as np

# Load the NumPy dataset
X = np.load('/content/X.npy')  # Assuming X.npy is the image data
y = np.load('/content/y.npy')  # Assuming y.npy is the label data


In [10]:
from sklearn.model_selection import train_test_split

# Normalize the images to [0, 1]
X = X / 255.0

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [13]:
print(X_train.shape)  # It should print something like (num_samples, 128, 128, 3)
print(X_val.shape)    # It should print something like (num_samples, 128, 128, 3)


(4006, 128, 128, 3)
(858, 128, 128, 3)


In [11]:
from tensorflow.keras import layers, models

def build_capsule_network(input_shape=(128, 128, 3), num_classes=4):
    input_layer = layers.Input(shape=input_shape)

    # CNN layers for feature extraction
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Capsule layers
    primary_capsules = layers.Conv2D(256, (9, 9), strides=2, padding='valid', activation='relu')(x)
    primary_capsules = layers.Reshape((-1, 8))(primary_capsules)

    # Capsule Layer
    caps_layer = layers.Dense(128, activation='sigmoid')(primary_capsules)
    caps_layer = layers.Reshape((-1, 16))(caps_layer)

    # Decoder network for the output
    decoder_input = layers.Input(shape=(16,))
    decoder = layers.Dense(512, activation='relu')(decoder_input)
    decoder = layers.Dense(1024, activation='relu')(decoder)
    output_layer = layers.Dense(num_classes, activation='softmax')(decoder)

    # Compile model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Build the model for 4 classes
model = build_capsule_network(num_classes=4)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │      8,704 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1024)      │    525,312 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 4)         │      4,100 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 538,116 (2.05 MB)

 Trainable params: 538,116 (2.05 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras import layers, models

def build_capsule_network(input_shape=(128, 128, 3), num_classes=4):
    input_layer = layers.Input(shape=input_shape)

    # CNN layers for feature extraction
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Capsule layer 1 - Primary Capsule
    primary_capsules = layers.Conv2D(256, (9, 9), strides=2, padding='valid', activation='relu')(x)
    primary_capsules = layers.Reshape((-1, 8))(primary_capsules)  # Flatten capsules, shape (batch_size, num_capsules, capsule_dim)

    # Capsule layer 2 - A fully connected layer of capsules
    caps_layer = layers.Dense(128, activation='sigmoid')(primary_capsules)
    caps_layer = layers.Reshape((-1, 16))(caps_layer)  # Ensure correct reshaping

    # Decoder network to predict the final class output
    decoder_input = layers.Input(shape=(16,))
    decoder = layers.Dense(512, activation='relu')(decoder_input)
    decoder = layers.Dense(1024, activation='relu')(decoder)
    output_layer = layers.Dense(num_classes, activation='softmax')(decoder)

    # Compile model
    model = models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Build the model for 4 classes
model = build_capsule_network(num_classes=4)
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 512)       │      8,704 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 1024)      │    525,312 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 4)         │      4,100 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 538,116 (2.05 MB)

 Trainable params: 538,116 (2.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras import layers, models

def build_simple_cnn(input_shape=(128, 128, 3), num_classes=4):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))  # For multi-class classification
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create a simple CNN model
simple_cnn_model = build_simple_cnn(input_shape=(128, 128, 3), num_classes=4)
simple_cnn_model.summary()

# Train the model
history = simple_cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 61, 61, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,821,892 (56.54 MB)

 Trainable params: 14,821,892 (56.54 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
 50/126 ━━━━━━━━━━━━━━━━━━━━ 2:19 2s/step - accuracy: 0.2475 - loss: 1.3867

In [18]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0



In [19]:
# Train the model
# history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m139322943189584\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(None, 128, 128, 3), dtype=float32)\n  • training=True\n  • mask=None'